# 회귀 기반 추천 시스템

> CatBoost (Categorical Boosting)

In [23]:
!pip install catboost

**데이터 구조**

![](https://d.pr/i/YEs8M6+)


여행스타일 8가지 (7단계)
(매우선호 - 중간선호 - 약간선호 - 중립 - 약간선호 - 중간선호 - 매우 선호)
- `TRAVEL_STYL_1` 자연 vs 도시
- `TRAVEL_STYL_2` 숙박 vs 당일
- `TRAVEL_STYL_3` 새로운 지역 vs 익숙한 지역
- `TRAVEL_STYL_4` 편하지만 비싼 숙소 vs 불편하지만 저렴한 숙소
- `TRAVEL_STYL_5` 휴양/휴식 vs 체험활동
- `TRAVEL_STYL_6` 잘알려지지 않은 방문지 vs 알려진 방문지 
- `TRAVEL_STYL_7` 계획에 따른 여행 vs 상황에 따른 여행
- `TRAVEL_STYL_8` 사진촬영 중요하지 않음 vs 사진촬영 중요

여행동기 3가지 (10단계)
- `TRAVEL_MOTIVE_1` 여행의 주요 목적
- `TRAVEL_MOTIVE_2` 여행의 부수적 목적1
- `TRAVEL_MOTIVE_3` 여행의 부수적 목적2
1. 일상적인 환경에서의 탈출
2. 육체적 정신적 휴식
3. 여행 동반자와의 친밀감 증진
4. 자아찾기
5. … (확인 안됨)

In [24]:
import pandas as pd

travel_df = pd.read_csv('./data/travel.csv')
# travel_df.shape
travel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34572 entries, 0 to 34571
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GENDER                 34572 non-null  object 
 1   AGE_GRP                34572 non-null  float64
 2   TRAVEL_STYL_1          34572 non-null  float64
 3   TRAVEL_STYL_2          34572 non-null  float64
 4   TRAVEL_STYL_3          34572 non-null  float64
 5   TRAVEL_STYL_4          34572 non-null  float64
 6   TRAVEL_STYL_5          34572 non-null  float64
 7   TRAVEL_STYL_6          34572 non-null  float64
 8   TRAVEL_STYL_7          34572 non-null  float64
 9   TRAVEL_STYL_8          34572 non-null  float64
 10  TRAVEL_MOTIVE_1        34572 non-null  float64
 11  TRAVEL_COMPANIONS_NUM  34572 non-null  float64
 12  VISIT_AREA_NM          34572 non-null  object 
 13  MVMN_NM                34572 non-null  object 
 14  DGSTFN                 34572 non-null  float64
dtypes:

In [25]:
# float 데이터 -> int 변환
travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']] \
    = travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']].astype(int)

# travel_df.head()

In [26]:
from sklearn.model_selection import train_test_split

X = travel_df.drop("DGSTFN", axis=1)
y = travel_df["DGSTFN"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [27]:
from catboost import Pool

# 범주형 특성 배열
cat_features = [
    'GENDER', 'AGE_GRP', 
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 
    'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 
    'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM',
    'VISIT_AREA_NM', 'MVMN_NM'
]

# CatBoost 모델에서의 사용을 위한 Pool 객체 생성
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

In [28]:
# 모델 생성 및 학습
from catboost import CatBoostRegressor

cb_reg = CatBoostRegressor(
    n_estimators=500,       # 반복횟수 (내부 예측기 개수)
    depth=5,                # 개별 트리의 최대 깊이
    learning_rate=0.03,     # 학습률
    loss_function='RMSE',   # 손실함수 (기본값)
    eval_metric='RMSE'      # 평가지표 (기본값) - 학습과 평가(검증) 동시 진행 지원
)

cb_reg.fit(train_pool, eval_set=test_pool, verbose=100)
# verbose=100 : 학습 100회당 한번씩 학습 및 검증 결과 출력

0:	learn: 0.8341962	test: 0.8359310	best: 0.8359310 (0)	total: 31.9ms	remaining: 15.9s
100:	learn: 0.7985342	test: 0.7931506	best: 0.7931506 (100)	total: 3.24s	remaining: 12.8s
200:	learn: 0.7863085	test: 0.7799891	best: 0.7799891 (200)	total: 6.61s	remaining: 9.84s
300:	learn: 0.7817498	test: 0.7763301	best: 0.7763301 (300)	total: 9.8s	remaining: 6.48s
400:	learn: 0.7787237	test: 0.7746257	best: 0.7746257 (400)	total: 13.1s	remaining: 3.23s
499:	learn: 0.7764294	test: 0.7733956	best: 0.7733956 (499)	total: 16.4s	remaining: 0us

bestTest = 0.773395606
bestIteration = 499



In [29]:
# 특성 중요도
col_importance = pd.DataFrame({
    'column': X_train.columns,
    'importance': cb_reg.feature_importances_
})

col_importance

,column,importance
0,GENDER,0.373389
1,AGE_GRP,7.351231
2,TRAVEL_STYL_1,7.058387
3,TRAVEL_STYL_2,6.958940
4,TRAVEL_STYL_3,4.288193
5,TRAVEL_STYL_4,9.604211
6,TRAVEL_STYL_5,8.382338
7,TRAVEL_STYL_6,8.360303
8,TRAVEL_STYL_7,8.541926
9,TRAVEL_STYL_8,11.937605


### 추천 시스템 구축

1. 방문지 목록을 생성
2. 사용자 특성 입력
3. 가상 만족도 예측
4. 만족도가 높은 순으로 추천

In [30]:
# 1. 방문지 목록 생성

visit_areas = travel_df['VISIT_AREA_NM'].unique()
# visit_areas[:10]

In [31]:
# 2. 사용자 특성 입력
user_input = ['여', 60, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, '방문지', '자가용']
pred_results = []

# 3. 가상 만족도 예측
for area in visit_areas:
    user_input[-2] = area
    dgstfn_pred = cb_reg.predict(user_input)
    pred_results.append(dgstfn_pred)

pred_results[:10]

[np.float64(4.338881865504073),
 np.float64(4.149058377045566),
 np.float64(4.33214199958774),
 np.float64(4.190207715847474),
 np.float64(4.147286100972279),
 np.float64(4.126341464044342),
 np.float64(4.093713467767728),
 np.float64(4.147286100972279),
 np.float64(4.2713833543500925),
 np.float64(4.2051950289527)]

In [32]:
# 4. 만족도가 높은 순으로 출력 (추천)
result_df = pd.DataFrame({
    'VISIT_AREA_NM': visit_areas,
    'DGSTFN_PRED': pred_results
})

result_df.sort_values(by='DGSTFN_PRED', ascending=False).head(10)

,VISIT_AREA_NM,DGSTFN_PRED
47,보래드베이커스,4.555464
3054,대정오일시장,4.549863
2023,호텔연,4.548418
824,호텔리젠트마린,4.546289
1485,청파식당횟집,4.546285
4749,제주스럽닭 서귀포올레점,4.544025
6434,알뜨르비행장,4.536824
129,스누피가든,4.536047
3448,막둥이해녀복순이네,4.536003
481,하라케케,4.533105
